In [ ]:
import pandas as pd
import numpy as np
import sklearn
import nltk
import re

In [ ]:
data = pd.read_csv("hn2018_2019.csv")

In [ ]:
def dater(datetim,integer):
    date,time = datetim.split(" ")
    year,month,day = date.split("-")
    hour,minute,second = time.split(":")
    second = int(second)+ int(minute)*60 + int(hour)*60*60
    if integer==1:
        return year
    elif integer==2:
        return month
    elif integer==3:
        return day
    else:
        return second

In [ ]:
data["year"] = data['Created At'].map(lambda x: dater(x,1))
data["month"] = data['Created At'].map(lambda x: dater(x,2))
data["day"] = data['Created At'].map(lambda x: dater(x,3))
data["second"] = data['Created At'].map(lambda x: dater(x,4))

In [ ]:
df_2018 = data[data["year"]=='2018'][['Title','Post Type']].copy()

In [ ]:
df_2018['Title'] = df_2018['Title'].map(lambda x:x.lower())

## Adding a column of tokenized words in the dataframe

In [ ]:
df_2018['tokenized_title'] = df_2018['Title'].map(lambda x:re.split('\[\^a-zA-Z\]',x)[0].split())

In [ ]:
data.head()

In [ ]:
df_2018.head()

## Calculating frequency of each word and given their conditional post type

In [ ]:
def input_phrase(phrase,typ,dict_words):
    for i in phrase:
        if i not in dict_words:
            dict_words[i] = {'story':0,'ask_hn':0,'show_hn':0,'poll':0}
        dict_words[i][typ] += 1
    return dict_words

In [ ]:
d = {}
num = 0
for i in range(len(df_2018['tokenized_title'])):
    d = input_phrase(df_2018['tokenized_title'][i],df_2018['Post Type'][i],d)
    num += len(df_2018['tokenized_title'][i])

In [ ]:
words_2018 = pd.DataFrame(d)
words_2018 = words_2018.transpose()
words_2018.index.name = 'TokenName'
words_2018 = words_2018.reset_index()
words_2018 = words_2018.sort_values(by ='TokenName')

In [ ]:
words_2018.head()

In [ ]:
delta = 0.5

In [ ]:
words_2018['prob_ask_hn'] = words_2018['ask_hn'].map(lambda x: (int(x)+delta)/(num+delta*len(words_2018)))
words_2018['prob_poll'] = words_2018['poll'].map(lambda x: (int(x)+delta)/(num+delta*len(words_2018)))
words_2018['prob_show_hn'] = words_2018['show_hn'].map(lambda x: (int(x)+delta)/(num+delta*len(words_2018)))
words_2018['prob_story'] = words_2018['story'].map(lambda x: (int(x)+delta)/(num+delta*len(words_2018)))

In [ ]:
words_2018.head()

In [ ]:
f = open("model-2018.txt","w+")
count = 0
for index in words_2018.index:
    line = str(count + 1) + '  '
    line += str(words_2018['TokenName'][index]) + '  '
    line += str(words_2018['story'][index]) + '  '
    line += str(words_2018['prob_story'][index]) + '  '
    line += str(words_2018['ask_hn'][index]) + '  '
    line += str(words_2018['prob_ask_hn'][index]) + '  '
    line += str(words_2018['show_hn'][index]) + '  '
    line += str(words_2018['prob_show_hn'][index]) + '  '
    line += str(words_2018['poll'][index]) + '  '
    line += str(words_2018['prob_poll'][index]) + '\n'
    f.write(line)
    count += 1
f.close()

## PART 2

In [ ]:
priorProbabilities = {'story':0,'ask_hn':0,'show_hn':0,'poll':0}
unique, counts = np.unique(df_2018['Post Type'], return_counts=True)
for index in range(len(unique)):
    priorProbabilities[unique[index]] = counts[index]/np.sum(counts)
print(priorProbabilities)

In [ ]:
print(len(words_2018))

In [ ]:
testData = data[data["year"]=='2019'][['Title','Post Type']].copy()
testData['Title'] = testData['Title'].map(lambda x:x.lower())
testData['tokenized_title'] = testData['Title'].map(lambda x:re.split('\[\^a-zA-Z\]',x)[0].split())

In [ ]:
testData.head()